In [68]:
cd("/workspaces/superVoxelJuliaCode_lin_sampl")


# In the Julia REPL:
using Pkg

Pkg.activate(".")  # Activate the current directory as project
# Pkg.add("Unitful")
# Pkg.instantiate()

using Revise
using Meshes
using LinearAlgebra
using GLMakie
using Combinatorics
using SplitApplyCombine
using CUDA
using Combinatorics
using Random
using Statistics
using ChainRulesCore
using Test
using Unitful # Often used in Meshes.jl, good practice to include
using LinearAlgebra # For norm, though Meshes.measure is preferred



using Logging
includet("/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/src/lin_sampl/sv_points/initialize_sv.jl")
includet("/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/src/lin_sampl/sv_points/points_from_weights.jl")


includet("/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/src/lin_sampl/loss_kerns/custom_kern _old.jl")
includet("/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/src/lin_sampl/utils_lin_sampl.jl")
includet("/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/src/lin_sampl/sv_points/sv_centr_from_weights.jl")
includet("/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/src/lin_sampl/sv_points/additional_points_from_weights.jl")
includet("/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/src/lin_sampl/loss_kerns/dif_custom_kern_tetr.jl")


Random.seed!(3)
is_point_per_triangle=true
global const num_additional_oblique_points_per_side=2

  Activating project at `/workspaces/superVoxelJuliaCode_lin_sampl`


2

## Selecting intial image and supervoxel values

In [69]:
#radius of supervoxel
radiuss = (Float32(3.1), Float32(4.3), Float32(4.7))
#size of each voxel - here we use isovolumetric voxels
spacing = (Float32(1.0), Float32(1.0), Float32(1.0))
#size of the image tensor including batch size; x,y,z dimensions are set to be a
batch_size = 2
a = 71
image_shape = (a, a, a, 2)
#we are setting weights shape - here making it too big for needs for convinience
weights_shape = Int.(round.((a / 2, a / 2, a / 2, 100)))

#we are initializing the grid of points (the regular ones) in sv_centers
#control_points - is place holder of control points for the SVs that are inferred from weights
#tetrs - is indicating indicating the division of supervoxels into tetrahedrons using the control points and sv_centers
example_set_of_svs = initialize_centers_and_control_points(image_shape, radiuss)
sv_centers, control_points, tetrs, dims = example_set_of_svs


(Float32[4.1 4.1 … 4.1 4.1; 10.299999 10.299999 … 10.299999 10.299999; … ; 59.899998 59.899998 … 59.899998 59.899998; 66.1 66.1 … 66.1 66.1;;; 4.1 4.1 … 4.1 4.1; 10.299999 10.299999 … 10.299999 10.299999; … ; 59.899998 59.899998 … 59.899998 59.899998; 66.1 66.1 … 66.1 66.1;;; 4.1 4.1 … 4.1 4.1; 10.299999 10.299999 … 10.299999 10.299999; … ; 59.899998 59.899998 … 59.899998 59.899998; 66.1 66.1 … 66.1 66.1;;; 4.1 4.1 … 4.1 4.1; 10.299999 10.299999 … 10.299999 10.299999; … ; 59.899998 59.899998 … 59.899998 59.899998; 66.1 66.1 … 66.1 66.1;;; 4.1 4.1 … 4.1 4.1; 10.299999 10.299999 … 10.299999 10.299999; … ; 59.899998 59.899998 … 59.899998 59.899998; 66.1 66.1 … 66.1 66.1;;; 4.1 4.1 … 4.1 4.1; 10.299999 10.299999 … 10.299999 10.299999; … ; 59.899998 59.899998 … 59.899998 59.899998; 66.1 66.1 … 66.1 66.1;;; 4.1 4.1 … 4.1 4.1; 10.299999 10.299999 … 10.299999 10.299999; … ; 59.899998 59.899998 … 59.899998 59.899998; 66.1 66.1 … 66.1 66.1;;;; 5.3 13.900001 … 56.9 65.50001; 5.3 13.900001 … 56.9 

In [70]:
#we copy it to get the same control points for all the batches (for testing)
control_points = repeat(control_points, inner=(1, 1, 1, 1, 1, batch_size))
#we are initializing the weights of the SVs a bit too big for convinience
weights_shape = Int.(round.((a / 2, a / 2, a / 2, 110, batch_size)))
rng = MersenneTwister(46)  # Using seed 42 for reproducible results

weights = rand(rng,weights_shape...) #ones(weights_shape).-0.5 #
#we initialize random image - in this case it do not influence the outcome as weights are not learned
image_shape = (a, a, a, 2, batch_size)
# Initialize a Mersenne Twister RNG with a specific seed for reproducibility
source_arr = rand(rng,image_shape...)
#cast to float32
weights = Float32.(weights)
control_points = Float32.(control_points)
source_arr = Float32.(source_arr)
sv_centers = Float32.(sv_centers)

#print sizes
print("\n  control_points $(size(control_points)) \n ")
print("\n  sv_centers $(size(sv_centers)) \n ")
print("\n  tetrs $(size(tetrs)) \n ")
print("\n  source_arr $(size(source_arr)) \n ")
print("\n  weights $(size(weights)) \n ")



  control_points (10, 7, 6, 16, 3, 2) 
 
  sv_centers (11, 8, 7, 3) 
 
  tetrs (32400, 5, 4) 
 
  source_arr (71, 71, 71, 2, 2) 
 
  weights (36, 36, 36, 110, 2) 
 

## main Grid
Visualizing first initial regular grid controlled by radius, and then how it changed after applying random weights to their locations by call_apply_weights_sv .

In [71]:
# sv_centers is our (nx, ny, nz, 3) tensor and we want center to visualize
function viz_sv_centers(sv_centers)
    dims = size(sv_centers)[1:3]
    center_indices = dims .÷ 2
    half_size = 2

    # Calculate start/end indices, clamping to tensor bounds
    start_indices = max.(1, center_indices .- half_size .+ 1)
    end_indices   = min.(dims, center_indices .+ half_size)

    # Extract the subset
    subset_coords = sv_centers[start_indices[1]:end_indices[1],
                            start_indices[2]:end_indices[2],
                            start_indices[3]:end_indices[3], :]

    # Reshape to easily iterate over points and create Point objects
    points_to_visualize = [Meshes.Point(subset_coords[i, j, k, :]...)
                        for i in axes(subset_coords, 1),
                            j in axes(subset_coords, 2),
                            k in axes(subset_coords, 3)] |> vec


    # Create figure and 3D axis
    fig = Figure()
    ax = Axis3(fig[1, 1], aspect = :data, title = "Central Subset Points")
    return fig, ax, points_to_visualize
end
# Visualize the points
fig, ax, points_to_visualize = viz_sv_centers(sv_centers)

viz!(ax, points_to_visualize, pointsize=10, color=:red)

# Display the plot
display(fig)                           

TaskFailedException: TaskFailedException

    nested task error: InterruptException:
    Stacktrace:
      [1] poptask(W::Base.IntrusiveLinkedListSynchronized{Task})
        @ Base ./task.jl:1012
      [2] wait()
        @ Base ./task.jl:1021
      [3] wait(c::Base.GenericCondition{Base.Threads.SpinLock}; first::Bool)
        @ Base ./condition.jl:130
      [4] wait
        @ ./condition.jl:125 [inlined]
      [5] _trywait(t::Timer)
        @ Base ./asyncevent.jl:145
      [6] wait
        @ ./asyncevent.jl:162 [inlined]
      [7] sleep(sec::Float64)
        @ Base ./asyncevent.jl:272
      [8] sleep(timer::Makie.BudgetedTimer)
        @ Makie /usr/local/share/julia/packages/Makie/pFPBw/src/utilities/timing.jl:120
      [9] on_demand_renderloop(screen::GLMakie.Screen{GLFW.Window})
        @ GLMakie /usr/local/share/julia/packages/GLMakie/xG4T9/src/screen.jl:946
     [10] renderloop(screen::GLMakie.Screen{GLFW.Window})
        @ GLMakie /usr/local/share/julia/packages/GLMakie/xG4T9/src/

In [72]:
#Here we are getting the constant values for the kernel  - like thread blocks etc.
threads_apply_w, blocks_apply_w, num_blocks_z_pure_sv, num_blocks_y_pure_sv = prepare_for_apply_weights_to_locs_kern(size(sv_centers), weights_shape, batch_size)
#casting data to CUDA and executing functions that changes the location of the sv centers not more than 0.75 times radius in each direction
sv_centers_out = call_apply_weights_sv(CuArray(sv_centers), CuArray(weights), radiuss, threads_apply_w, blocks_apply_w, num_blocks_z_pure_sv, batch_size, num_blocks_y_pure_sv,[])
CUDA.synchronize()

In [73]:
#visualizing after changes 
fig, ax, points_to_visualize = viz_sv_centers(Array(sv_centers_out[:,:,:,:,1]))#ignore batch

viz!(ax, points_to_visualize, pointsize=10, color=:red)

# Display the plot
display(fig)    

TaskFailedException: TaskFailedException

    nested task error: InterruptException:
    Stacktrace:
      [1] poptask(W::Base.IntrusiveLinkedListSynchronized{Task})
        @ Base ./task.jl:1012
      [2] wait()
        @ Base ./task.jl:1021
      [3] wait(c::Base.GenericCondition{Base.Threads.SpinLock}; first::Bool)
        @ Base ./condition.jl:130
      [4] wait
        @ ./condition.jl:125 [inlined]
      [5] _trywait(t::Timer)
        @ Base ./asyncevent.jl:145
      [6] wait
        @ ./asyncevent.jl:162 [inlined]
      [7] sleep(sec::Float64)
        @ Base ./asyncevent.jl:272
      [8] sleep(timer::Makie.BudgetedTimer)
        @ Makie /usr/local/share/julia/packages/Makie/pFPBw/src/utilities/timing.jl:120
      [9] on_demand_renderloop(screen::GLMakie.Screen{GLFW.Window})
        @ GLMakie /usr/local/share/julia/packages/GLMakie/xG4T9/src/screen.jl:946
     [10] renderloop(screen::GLMakie.Screen{GLFW.Window})
        @ GLMakie /usr/local/share/julia/packages/GLMakie/xG4T9/src/

Displaying the original control sv center new one and bounding box indicating radiuss in each side 

In [74]:

"""
    visualize_centers_and_box(sv_center_orig, sv_center_new, radiuss)

Visualizes two center points and a box defined by the original center and radii.

- `sv_center_orig`: Vector [x,y,z] for the original center (box center).
- `sv_center_new`: Vector [x,y,z] for the new center.
- `radiuss`: Tuple (rx, ry, rz) defining half-dimensions of the box.
- `to_viz`: Boolean flag to control visualization.
"""
function visualize_centers_and_box(sv_center_orig::AbstractVector, sv_center_new::AbstractVector, radiuss::NTuple{3,<:Real},to_viz=true)
    # Ensure GLMakie is the active backend
    GLMakie.activate!()

    # Convert input vectors to Meshes.Point
    p_orig = Meshes.Point(sv_center_orig...)
    p_new  = Meshes.Point(sv_center_new...)

    # Extract radii as Float64
    rx, ry, rz = Float64.(radiuss)

    # Calculate box corner coordinates relative to sv_center_orig
    half_sides = [rx, ry, rz]
    min_corner_coords = sv_center_orig .- (half_sides.*2)
    max_corner_coords = sv_center_orig .+ (half_sides.*2) # Corrected calculation

    # Define the 8 vertices of the box as Meshes.Point
    min_c = min_corner_coords
    max_c = max_corner_coords
    v1 = Meshes.Point(min_c[1], min_c[2], min_c[3])
    v2 = Meshes.Point(max_c[1], min_c[2], min_c[3])
    v3 = Meshes.Point(max_c[1], max_c[2], min_c[3])
    v4 = Meshes.Point(min_c[1], max_c[2], min_c[3])
    v5 = Meshes.Point(min_c[1], min_c[2], max_c[3])
    v6 = Meshes.Point(max_c[1], min_c[2], max_c[3])
    v7 = Meshes.Point(max_c[1], max_c[2], max_c[3])
    v8 = Meshes.Point(min_c[1], max_c[2], max_c[3])

    # Define the 12 edges (segments) of the box
    box_edges = [
        Meshes.Segment(v1, v2), Meshes.Segment(v2, v3), Meshes.Segment(v3, v4), Meshes.Segment(v4, v1), # Bottom
        Meshes.Segment(v5, v6), Meshes.Segment(v6, v7), Meshes.Segment(v7, v8), Meshes.Segment(v8, v5), # Top
        Meshes.Segment(v1, v5), Meshes.Segment(v2, v6), Meshes.Segment(v3, v7), Meshes.Segment(v4, v8)  # Vertical
    ]

    # Create a figure and 3D axis
    fig = Figure()
    ax = Axis3(fig[1, 1], aspect = :data, title = "Supervoxel Centers and Box")

    # Visualize points and box edges
    viz!(ax, p_orig, color = :red, pointsize = 15)
    viz!(ax, p_new, color = :green, pointsize = 15)
    if(to_viz)
        # Use the viz! method for plotting geometries from Meshes.jl
        viz!(ax, box_edges, color = :black) # segmentsize is not a direct attribute for viz! on segments, color works

        # Display the plot
        display(fig)
    end    

    return fig,ax,box_edges,p_orig,p_new # Optionally return the figure object
end
# fig,ax,box_edges,p_orig,p_new=visualize_centers_and_box(Array(sv_centers[2,2,2,:]), Array(sv_centers_out[2,2,2,:,1]), radiuss)

visualize_centers_and_box

## Getting main control points 
First we get the control constants like number of threadblocks etc for executing main control points kernel and then we execute it. We start from getting linear control points so in x,y,z axis between current and next in axis sv center and then main oblique so in corners shared by 6 supervoxels 
lin_x y z is in first 3 channels of supervoxels and main oblique in 4th channel

In [75]:
#first we get the control constants like number of threadblocks etc for executing main control points kernel and then we execute it 

threads_apply_w, blocks_apply_w, num_blocks_z_pure, num_blocks_y_pure_w = prepare_for_apply_weights_to_locs_kern(size(control_points), weights_shape, batch_size)
# control_points=Float32.(control_points)
control_points_out = call_apply_weights_to_locs_kern(sv_centers_out, size(control_points), CuArray(weights), threads_apply_w, blocks_apply_w, num_blocks_z_pure, num_blocks_y_pure_w,[])
CUDA.synchronize()


In [76]:

"""
Visualizes centers, box, and control points, returning plot handles and meshes.

Args:
    main_index (Tuple{Int,Int,Int}): Index (ix, iy, iz) for sv_centers.
    sv_centers (Array{T, 4}): Tensor of original supervoxel centers.
    sv_centers_out (Array{T, 5}): Tensor of new supervoxel centers.
    radiuss (NTuple{3, <:Real}): Half-dimensions of the bounding box.
    control_points_out (Array{T, 6}): Tensor of control points.

Returns:
    Tuple: (fig, ax, all_meshes) where all_meshes is a Dict mapping names to Meshes.jl objects.
"""
function visualize_all_points_and_return_meshes(main_index::Tuple{Int,Int,Int}, sv_centers, sv_centers_out, radiuss, control_points_out)
    # --- Step 1: Visualize centers and box using the helper function ---
    println("Step 1: Visualizing centers and bounding box...")
    center_orig_coords = Array(sv_centers[main_index..., :])
    center_new_coords  = Array(sv_centers_out[main_index..., :, 1]) # Batch index 1

    # Call the helper function to get the figure, axis, and initial meshes
    fig, ax, box_edges, p_orig, p_new = visualize_centers_and_box(center_orig_coords, center_new_coords, radiuss,false)

    # Initialize dictionary to store all created Meshes.jl objects
    all_meshes = Dict{String, Any}(
        "original_center" => p_orig,
        "new_center" => p_new,
        "bounding_box_edges" => box_edges
    )

    # --- Step 2: Prepare indices for control points ---
    println("Step 2: Preparing indices for control points...")
    # Control points are indexed starting from main_index - 1
    shifted_index = main_index .- 1
    ix_s, iy_s, iz_s = shifted_index

    # --- Step 3: Process lin_x control points ---
    println("Step 3: Processing lin_x control points...")
    # Point type 1 = lin_x
    lin_x_coords1 = Array(control_points_out[ix_s,   iy_s, iz_s, 1, :, 1])
    lin_x_coords2 = Array(control_points_out[ix_s+1, iy_s, iz_s, 1, :, 1])
    # Create Meshes objects
    lin_x_p1 = Meshes.Point(lin_x_coords1...)
    lin_x_p2 = Meshes.Point(lin_x_coords2...)
    lin_x_segment = Meshes.Segment(lin_x_p1, lin_x_p2)
    # Store meshes
    all_meshes["lin_x_points"] = [lin_x_p1, lin_x_p2]
    all_meshes["lin_x_segment"] = lin_x_segment
    # Plot meshes onto the existing axis `ax`
    viz!(ax, [lin_x_p1, lin_x_p2], color = :blue, pointsize = 10, label="lin_x")
    viz!(ax, lin_x_segment, color=:lightblue) # Use a lighter color for the segment

    # --- Step 4: Process lin_y control points ---
    println("Step 4: Processing lin_y control points...")
    # Point type 2 = lin_y
    lin_y_coords1 = Array(control_points_out[ix_s, iy_s,   iz_s, 2, :, 1])
    lin_y_coords2 = Array(control_points_out[ix_s, iy_s+1, iz_s, 2, :, 1])
    # Create Meshes objects
    lin_y_p1 = Meshes.Point(lin_y_coords1...)
    lin_y_p2 = Meshes.Point(lin_y_coords2...)
    lin_y_segment = Meshes.Segment(lin_y_p1, lin_y_p2)
    # Store meshes
    all_meshes["lin_y_points"] = [lin_y_p1, lin_y_p2]
    all_meshes["lin_y_segment"] = lin_y_segment
    # Plot meshes
    viz!(ax, [lin_y_p1, lin_y_p2], color = :cyan, pointsize = 10, label="lin_y")
    viz!(ax, lin_y_segment, color=:cyan4)

    # --- Step 5: Process lin_z control points ---
    println("Step 5: Processing lin_z control points...")
    # Point type 3 = lin_z
    lin_z_coords1 = Array(control_points_out[ix_s, iy_s, iz_s,   3, :, 1])
    lin_z_coords2 = Array(control_points_out[ix_s, iy_s, iz_s+1, 3, :, 1])
    # Create Meshes objects
    lin_z_p1 = Meshes.Point(lin_z_coords1...)
    lin_z_p2 = Meshes.Point(lin_z_coords2...)
    lin_z_segment = Meshes.Segment(lin_z_p1, lin_z_p2)
    # Store meshes
    all_meshes["lin_z_points"] = [lin_z_p1, lin_z_p2]
    all_meshes["lin_z_segment"] = lin_z_segment
    # Plot meshes
    viz!(ax, [lin_z_p1, lin_z_p2], color = :purple, pointsize = 10, label="lin_z")
    viz!(ax, lin_z_segment, color=:purple4)

    # --- Step 6: Process main_oblique control points ---
    println("Step 6: Processing main_oblique control points...")
    # Point type 4 = main_oblique
    main_oblique_points = Meshes.Point[]
    # Iterate through the 8 neighbors (including the starting one at dx=dy=dz=0)
    for dx in 0:1, dy in 0:1, dz in 0:1
        coords = Array(control_points_out[ix_s+dx, iy_s+dy, iz_s+dz, 4, :, 1])
        p = Meshes.Point(coords...)
        push!(main_oblique_points, p)
    end
    # Store meshes
    all_meshes["main_oblique_points"] = main_oblique_points
    # Plot meshes
    viz!(ax, main_oblique_points, color = :orange, pointsize = 12, label="main_oblique")

    # --- Final Step: Add legend and return ---
    println("Final Step: Adding legend and returning results...")
    # Add a legend to the axis to identify plotted elements by their labels
    axislegend(ax)
    # Display the final figure with all elements
    display(fig)
    # Return the figure, axis, and the dictionary of meshes
    return fig, ax, all_meshes
end


visualize_all_points_and_return_meshes

In [77]:
# Define the main index to visualize
main_index = (4, 4, 4)

# Call the visualization function"
fig, ax, meshes_dict = visualize_all_points_and_return_meshes(main_index, sv_centers, sv_centers_out, radiuss, control_points_out)

# You can now access the created meshes:
println("\nReturned Meshes Objects:")
for (key, value) in meshes_dict
    println("- $key: $(typeof(value))")
end

Step 1: Visualizing centers and bounding box...
Step 2: Preparing indices for control points...
Step 3: Processing lin_x control points...
Step 4: Processing lin_y control points...
Step 5: Processing lin_z control points...
Step 6: Processing main_oblique control points...
Final Step: Adding legend and returning results...
Step 2: Preparing indices for control points...
Step 3: Processing lin_x control points...
Step 4: Processing lin_y control points...
Step 5: Processing lin_z control points...
Step 6: Processing main_oblique control points...
Final Step: Adding legend and returning results...


TaskFailedException: TaskFailedException

    nested task error: InterruptException:
    Stacktrace:
      [1] poptask(W::Base.IntrusiveLinkedListSynchronized{Task})
        @ Base ./task.jl:1012
      [2] wait()
        @ Base ./task.jl:1021
      [3] wait(c::Base.GenericCondition{Base.Threads.SpinLock}; first::Bool)
        @ Base ./condition.jl:130
      [4] wait
        @ ./condition.jl:125 [inlined]
      [5] _trywait(t::Timer)
        @ Base ./asyncevent.jl:145
      [6] wait
        @ ./asyncevent.jl:162 [inlined]
      [7] sleep(sec::Float64)
        @ Base ./asyncevent.jl:272
      [8] sleep(timer::Makie.BudgetedTimer)
        @ Makie /usr/local/share/julia/packages/Makie/pFPBw/src/utilities/timing.jl:120
      [9] on_demand_renderloop(screen::GLMakie.Screen{GLFW.Window})
        @ GLMakie /usr/local/share/julia/packages/GLMakie/xG4T9/src/screen.jl:946
     [10] renderloop(screen::GLMakie.Screen{GLFW.Window})
        @ GLMakie /usr/local/share/julia/packages/GLMakie/xG4T9/src/

# Get additional control points 
We will now define additional control points that increase possibilities of upervoxels to adapt their shapes to the image . 
Main idea is to first deine shape in which we are looking for new ponts and that will meet our initial criteria - we want star polyhedron so all lines from sv_center to any point on the surface will be inside the supervoxel.
as we are sharing points between supervoxels it means that points need to at once comply with this requirement for set of sv_centers. 
now we will create new points by defining new shape that will look like two uneven pyramids joined by their bases; on the corner of the base are 4 supervoxel centers and in the apicies main oblique points current and next in given axis (we repeat it for each axis) 
We look for series of points in this shape that each of the point will be progressively moving from closest to current main oblique to the next main oblique ; number of new points per axis in single calculation will be "num_additional_oblique_points_per_side*2" where "num_additional_oblique_points_per_side" can be set depending on the needs.



For visualization we will concentrate at one area at a time that is in this double pyramid area ; and get the new points starting from x axis

In [78]:
#first we execute code to augment points with additional oblique points 

control_points_out = call_apply_weights_to_locs_kern_add_a(sv_centers_out, control_points_out, CuArray(weights), threads_apply_w, blocks_apply_w, num_blocks_z_pure, num_blocks_y_pure_w,[])
CUDA.synchronize()


In [79]:
"""
Visualizes specific points and connections based on a main index and axis.

Args:
    main_index (Tuple{Int,Int,Int}): Index (ix, iy, iz) for sv_centers and control_points_out.
    sv_centers (Array): Tensor of original supervoxel centers (used for box centering).
    sv_centers_out (Array): Tensor of new supervoxel centers.
    radiuss (NTuple{3, <:Real}): Half-dimensions of the bounding box.
    control_points_out (Array): Tensor of control points.
    axis (Int): The axis to focus on (1 for x, 2 for y, 3 for z).

Returns:
    Tuple: (fig, ax, all_meshes) where all_meshes is a Dict mapping names to Meshes.jl objects.
"""
function visualize_axis_points_only(main_index::Tuple{Int,Int,Int}, sv_centers, sv_centers_out, radiuss, control_points_out, axis::Int)
    # Define consistent type for Meshes.Point coordinates (Fixes the MethodError)
    T = Float64

    # --- Step 1: Initial Visualization (New Center and Box) ---
    println("Step 1: Visualizing new center and bounding box...")
    # Ensure coordinates passed to the helper are Float64
    center_orig_coords = T.(Array(sv_centers[main_index..., :]))
    center_new_coords  = T.(Array(sv_centers_out[main_index..., :, 1])) # Batch index 1

    # Call the helper function (setting to_viz=false to prevent double display)
    # It returns: fig, ax, box_edges (Vector{Segment}), p_orig (Point), p_new (Point)
    fig, ax, box_edges, p_orig, p_new = visualize_centers_and_box(center_orig_coords, center_new_coords, radiuss, false) # Pass false for to_viz

    # Re-visualize the new center point with double size and add label
    viz!(ax, p_new, color = :green, pointsize = 30, label="New Center") # Increased size and added label

    # Initialize dictionary to store meshes
    all_meshes = Dict{String, Any}(
        "original_center_coords" => center_orig_coords, # Store coords for reference if needed
        "new_center" => p_new, # Already Float64 from visualize_centers_and_box
        "bounding_box_edges" => box_edges # Already Float64 from visualize_centers_and_box
    )

    # --- Step 2: Prepare Indices ---
    println("Step 2: Preparing indices...")
    # Use main_index directly for control_points_out as per updated requirement
    current_index = main_index.-1
    ix_s, iy_s, iz_s = current_index

    # --- Step 2.5: Extract and Visualize lin_y and lin_z at current_index ---
    println("Step 2.5: Processing lin_y and lin_z at current index...")
    lin_y_type = 2
    lin_z_type = 3
    color_lin_y = :cyan
    color_lin_z = :purple

    # Extract lin_y
    if checkbounds(Bool, control_points_out, ix_s, iy_s, iz_s, lin_y_type, :, 1)
        lin_y_coords = T.(Array(control_points_out[ix_s, iy_s, iz_s, lin_y_type, :, 1]))
        lin_y_p = Meshes.Point(lin_y_coords...)
        all_meshes["lin_y_current"] = lin_y_p
        viz!(ax, lin_y_p, color = color_lin_y, pointsize = 11, label="lin_y (current)")
        println("  Added lin_y point.")
    else
        println("Warning: Indices out of bounds for lin_y at current index.")
    end

    # Extract lin_z
    if checkbounds(Bool, control_points_out, ix_s, iy_s, iz_s, lin_z_type, :, 1)
        lin_z_coords = T.(Array(control_points_out[ix_s, iy_s, iz_s, lin_z_type, :, 1]))
        lin_z_p = Meshes.Point(lin_z_coords...)
        all_meshes["lin_z_current"] = lin_z_p
        viz!(ax, lin_z_p, color = color_lin_z, pointsize = 11, label="lin_z (current)")
        println("  Added lin_z point.")
    else
        println("Warning: Indices out of bounds for lin_z at current index.")
    end


    # --- Step 3: Define Axis-Specific Data ---
    println("Step 3: Defining axis-specific data for axis = $axis...")
    # Validate axis input
    if !(axis in 1:3)
        error("Invalid axis integer. Use 1, 2, or 3.")
    end
    axis_int_key = axis # Use the integer axis directly

    # Define distinct colors for visualization
    # color_new_center = :green         # Set above
    color_main_oblique = :darkorange
    color_neighbor_center = :firebrick
    color_neighbor_lines = :lightcoral # Color for lines between neighbors

    # Define colors and labels for the 4 specific oblique points per axis
    # Using a nested structure for clarity
    specific_oblique_details = Dict(
        1 => Dict(zip(5:8, [(:red, "oblique_x_a"), (:blue, "oblique_x_b"), (:purple, "oblique_x_c"), (:yellow, "oblique_x_d")])), # Axis 1 (x)
        2 => Dict(zip(9:12, [(:tomato, "oblique_y_a"), (:cyan, "oblique_y_b"), (:violet, "oblique_y_c"), (:gold, "oblique_y_d")])), # Axis 2 (y) - slightly different colors
        3 => Dict(zip(13:16, [(:darkred, "oblique_z_a"), (:deepskyblue, "oblique_z_b"), (:indigo, "oblique_z_c"), (:khaki, "oblique_z_d")])) # Axis 3 (z) - more different colors
    )

    # Use integer keys 1, 2, 3 for the axis_data dictionary
    # Note: sv_centers_mod (neighbor_offsets) indices are shifted by +1 compared to control_points indices.
    # hence 1,1,1 is really 0,0,0 so indicating the same point when we use indicies of control points out
    axis_data = Dict(
        1 => ( # Corresponds to axis 1 (x)
            axis_idx = 1,
            axis_offset = (1, 0, 0),
            oblique_types = 5:8, # oblique_x_a to oblique_x_d
            neighbor_offsets = [(1, 1, 1), (1, 1, 0), (1, 0, 0), (1, 0, 1)], # Order matters for line drawing
            specific_details = specific_oblique_details[1] # Details for axis 1
        ),
        2 => ( # Corresponds to axis 2 (y)
            axis_idx = 2,
            axis_offset = (0, 1, 0),
            oblique_types = 9:12, # oblique_y_a to oblique_y_d
            neighbor_offsets = [(1, 1, 1), (1, 1, 0), (0, 1, 0), (0, 1, 1)], # Order matters
            specific_details = specific_oblique_details[2] # Details for axis 2
        ),
        3 => ( # Corresponds to axis 3 (z)
            axis_idx = 3,
            axis_offset = (0, 0, 1),
            oblique_types = 13:16, # oblique_z_a to oblique_z_d
            neighbor_offsets = [(1, 1, 1), (1, 0, 1), (0, 0, 1), (0, 1, 1)], # Order matters
            specific_details = specific_oblique_details[3] # Details for axis 3
        )
    )

    data = axis_data[axis_int_key] # Use the integer key
    oblique_point_types = data.oblique_types
    neighbor_index_offsets = data.neighbor_offsets
    axis_offset_tuple = data.axis_offset
    point_details_map = data.specific_details # Map from type index to (color, label)

    # --- Step 4: Visualize Main Obliques (2 points along the axis) ---
    println("Step 4: Processing 2 Main Oblique points...")
    main_oblique_type = 4
    # Indices are based on the *current* index
    main_oblique_idx1 = current_index
    main_oblique_idx2 = current_index .+ axis_offset_tuple

    # Ensure coordinates are Float64 before creating Points
    main_oblique_coords1 = T.(Array(control_points_out[main_oblique_idx1..., main_oblique_type, :, 1]))
    main_oblique_coords2 = T.(Array(control_points_out[main_oblique_idx2..., main_oblique_type, :, 1]))

    main_oblique_p1 = Meshes.Point(main_oblique_coords1...)
    main_oblique_p2 = Meshes.Point(main_oblique_coords2...)
    displayed_main_obliques = [main_oblique_p1, main_oblique_p2]

    all_meshes["main_obliques_$(axis)"] = displayed_main_obliques
    # Use the defined color and label for the legend
    viz!(ax, displayed_main_obliques, color = color_main_oblique, pointsize = 12, label="Main Obliques ($axis)")

    # --- Step 5: Visualize Specific Obliques (4 points for the axis, individually colored) ---
    println("Step 5: Processing 4 Specific Oblique points for axis $axis...")
    specific_oblique_points_list = Meshes.Point[] # Keep a list if needed later
    for pt_type in oblique_point_types
        # Access control points at the *current* index for these types
        # Ensure coordinates are Float64
        coords = T.(Array(control_points_out[ix_s, iy_s, iz_s, pt_type, :, 1]))
        p = Meshes.Point(coords...)
        push!(specific_oblique_points_list, p)

        # Get color and label for this specific point type
        point_color, point_label = point_details_map[pt_type]

        # Store individual point in the dictionary with its specific label
        all_meshes[point_label] = p

        # Plot each point individually with its unique color and label
        viz!(ax, p, color = point_color, pointsize = 10, label=point_label)
    end
    # Store the list as well, if needed for other operations
    all_meshes["specific_obliques_$(axis)_list"] = specific_oblique_points_list

    # --- Step 6: Visualize Neighboring Supervoxel Centers (4 points) and Connecting Lines ---
    println("Step 6: Processing 4 Neighboring Supervoxel Centers and connecting lines...")
    neighbor_sv_centers = Meshes.Point[]
    for offset in neighbor_index_offsets # Use the defined order
        # Neighbor indices are relative to the *original* main_index
        neighbor_idx = current_index .+ offset
        # Basic bounds check (optional, adjust as needed)
        if all(1 .<= neighbor_idx .<= size(sv_centers_out)[1:3])
            # Ensure coordinates are Float64
            coords = T.(Array(sv_centers_out[neighbor_idx..., :, 1]))
            p = Meshes.Point(coords...)
            push!(neighbor_sv_centers, p)
        else
            println("Warning: Neighbor index $neighbor_idx for sv_centers_out is out of bounds.")
        end
    end

    all_meshes["neighbor_sv_centers_$(axis)"] = neighbor_sv_centers
    # Use the defined color and label for the legend
    viz!(ax, neighbor_sv_centers, color = color_neighbor_center, pointsize = 13, label="Neighbor Centers ($axis)")

    # Add lines connecting the neighbor centers if we have exactly 4
    if length(neighbor_sv_centers) == 4
        neighbor_segments = [
            Meshes.Segment(neighbor_sv_centers[1], neighbor_sv_centers[2]),
            Meshes.Segment(neighbor_sv_centers[2], neighbor_sv_centers[3]),
            Meshes.Segment(neighbor_sv_centers[3], neighbor_sv_centers[4]),
            Meshes.Segment(neighbor_sv_centers[4], neighbor_sv_centers[1]) # Close the loop
        ]
        all_meshes["neighbor_connecting_segments_$(axis)"] = neighbor_segments
        viz!(ax, neighbor_segments, color=color_neighbor_lines, linewidth=2) # Visualize segments
        println("  Added connecting lines between neighbor centers.")
    else
        println("Warning: Did not find exactly 4 neighbor centers (found $(length(neighbor_sv_centers))). Skipping connecting lines.")
    end

    # --- Final Step: Add legend and return ---
    println("Final Step: Adding legend and returning results...")
    # Generate the legend based on the labels provided in viz! calls
    axislegend(ax, position = :rt) # Position legend top right
    display(fig) # Display the completed figure
    return fig, ax, all_meshes
end

# Call the visualization function
axis_to_visualize=1

fig, ax, meshes_dict = visualize_axis_points_only(main_index, Array(sv_centers), Array(sv_centers_out), radiuss, Array(control_points_out), axis_to_visualize)

# You can now access the created meshes:
println("\nReturned Meshes Objects for axis = $axis_to_visualize:")
for (key, value) in meshes_dict
    println("- $key: $(typeof(value))")
end

Step 1: Visualizing new center and bounding box...
Step 2: Preparing indices...
Step 2.5: Processing lin_y and lin_z at current index...
  Added lin_y point.
  Added lin_z point.
Step 3: Defining axis-specific data for axis = 1...
Step 4: Processing 2 Main Oblique points...
Step 5: Processing 4 Specific Oblique points for axis 1...
Step 6: Processing 4 Neighboring Supervoxel Centers and connecting lines...
  Added connecting lines between neighbor centers.
Final Step: Adding legend and returning results...
Step 2: Preparing indices...
Step 2.5: Processing lin_y and lin_z at current index...
  Added lin_y point.
  Added lin_z point.
Step 3: Defining axis-specific data for axis = 1...
Step 4: Processing 2 Main Oblique points...
Step 5: Processing 4 Specific Oblique points for axis 1...
Step 6: Processing 4 Neighboring Supervoxel Centers and connecting lines...
  Added connecting lines between neighbor centers.
Final Step: Adding legend and returning results...


TaskFailedException: TaskFailedException

    nested task error: InterruptException:
    Stacktrace:
      [1] poptask(W::Base.IntrusiveLinkedListSynchronized{Task})
        @ Base ./task.jl:1012
      [2] wait()
        @ Base ./task.jl:1021
      [3] wait(c::Base.GenericCondition{Base.Threads.SpinLock}; first::Bool)
        @ Base ./condition.jl:130
      [4] wait
        @ ./condition.jl:125 [inlined]
      [5] _trywait(t::Timer)
        @ Base ./asyncevent.jl:145
      [6] wait
        @ ./asyncevent.jl:162 [inlined]
      [7] sleep(sec::Float64)
        @ Base ./asyncevent.jl:272
      [8] sleep(timer::Makie.BudgetedTimer)
        @ Makie /usr/local/share/julia/packages/Makie/pFPBw/src/utilities/timing.jl:120
      [9] on_demand_renderloop(screen::GLMakie.Screen{GLFW.Window})
        @ GLMakie /usr/local/share/julia/packages/GLMakie/xG4T9/src/screen.jl:946
     [10] renderloop(screen::GLMakie.Screen{GLFW.Window})
        @ GLMakie /usr/local/share/julia/packages/GLMakie/xG4T9/src/

## Parsing the points into tetrahedrons
We will use another kernel to parse all of the points into tetrahedron format (so we will repeat points separately ofor each tetrahedron that is constituing part of the supervoxel)

In [80]:
threads_tetr_set, blocks_tetr_set = prepare_for_set_tetr_dat(image_shape, size(tetrs), batch_size)
tetr_dat_out = call_set_tetr_dat_kern(CuArray(tetrs), CuArray(source_arr), control_points_out, sv_centers_out, threads_tetr_set, blocks_tetr_set, spacing, batch_size)
CUDA.synchronize()


Get some helper functions so we can get triangles of the bases of tetrahedrons in that way visualizing the supervoxel surface

In [81]:

function fill_tetrahedron_data(tetr_dat, sv_centers, control_points, index)
    center = map(axis -> sv_centers[Int(tetr_dat[index, 1, 1]), Int(tetr_dat[index, 1, 2]), Int(tetr_dat[index, 1, 3]), axis], [1, 2, 3])
    corners = map(corner_num ->
            map(axis -> control_points[Int(tetr_dat[index, corner_num, 1]), Int(tetr_dat[index, corner_num, 2]), Int(tetr_dat[index, corner_num, 3]), Int(tetr_dat[index, corner_num, 4]), axis], [1, 2, 3]), [2, 3, 4])
    corners = [center, corners...]
    return corners
end

function get_tetrahedrons_from_corners(corners)
    points = map(el -> Meshes.Point((el[1], el[2], el[3])), corners)
    return Meshes.Tetrahedron(points...)
end
function get_base_triangles_from_corners(corners)
    points = map(el -> Meshes.Point((el[1], el[2], el[3])), corners)
    return Meshes.Triangle(points[2:end]...)
end


function get_tetr_for_vis(curr_batch, n, tetr_dat_out, sv_centers_out, control_points_out, tetrs)
    size(sv_centers_out)
    size(sv_centers_out)
    size(control_points_out)
    # tetrs=tetrs[:,:,:,curr_batch]
    sv_centers_out = sv_centers_out[:, :, :, :, curr_batch]
    control_points_out = control_points_out[:, :, :, :, :, curr_batch]
    first_sv_tetrs_a = map(index -> fill_tetrahedron_data(tetrs, Array(sv_centers_out), Array(control_points_out), index), ((get_num_tetr_in_sv(is_point_per_triangle)*n)+1):get_num_tetr_in_sv(is_point_per_triangle)*(n+2))
    first_sv_tetrs = map(get_tetrahedrons_from_corners, first_sv_tetrs_a)
    first_sv_tetrs_base_triangles = map(get_base_triangles_from_corners, first_sv_tetrs_a)

    # curr=first_sv_tetrs_base_triangles[24:30]
    curr = first_sv_tetrs_base_triangles[1:get_num_tetr_in_sv(false)]
    # curr = first_sv_tetrs_base_triangles[37:48]
    # curr = first_sv_tetrs_base_triangles
    return curr
end




get_tetr_for_vis (generic function with 1 method)

In [82]:

curr = get_tetr_for_vis(2, 1, tetr_dat_out, sv_centers_out, control_points_out, tetrs)

curr
# curr=first_sv_tetrs_base_triangles[1:12]
# curr=first_sv_tetrs_base_triangles[37:48]
# curr=first_sv_tetrs_base_triangles
# viz(curr,color=[2,2,2,2,ones(8)...,3,3,3,3,ones(8)...] )#,alpha=0.5
# viz(curr,color=[2,2,2,2,ones(length(curr)-4)...] )#,alpha=0.5
curr_b = []

for i in 1:length(curr)
    try
        viz(curr[i])
        push!(curr_b, curr[i])
    catch e
        println("Error visualizing curr[$i]: $e")
    end
end

viz(curr_b, color=1:length(curr_b))#,alpha=0.5

TaskFailedException: TaskFailedException

    nested task error: InterruptException:
    Stacktrace:
      [1] poptask(W::Base.IntrusiveLinkedListSynchronized{Task})
        @ Base ./task.jl:1012
      [2] wait()
        @ Base ./task.jl:1021
      [3] wait(c::Base.GenericCondition{Base.Threads.SpinLock}; first::Bool)
        @ Base ./condition.jl:130
      [4] wait
        @ ./condition.jl:125 [inlined]
      [5] _trywait(t::Timer)
        @ Base ./asyncevent.jl:145
      [6] wait
        @ ./asyncevent.jl:162 [inlined]
      [7] sleep(sec::Float64)
        @ Base ./asyncevent.jl:272
      [8] sleep(timer::Makie.BudgetedTimer)
        @ Makie /usr/local/share/julia/packages/Makie/pFPBw/src/utilities/timing.jl:120
      [9] on_demand_renderloop(screen::GLMakie.Screen{GLFW.Window})
        @ GLMakie /usr/local/share/julia/packages/GLMakie/xG4T9/src/screen.jl:946
     [10] renderloop(screen::GLMakie.Screen{GLFW.Window})
        @ GLMakie /usr/local/share/julia/packages/GLMakie/xG4T9/src/

## Additional test
Generally in order to be sure we have valid star polyhedrons we need to be sure that segment from the sv center to any control point of this supervoxel is contained in this supervoxel

In [83]:
includet("/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/src/tests/points_from_weights/check_is_star.jl")
check_star_shape(Array(tetr_dat_out))

Checking 270 supervoxels for star-shape property (batch index 1)...
-------------------------------------------
Error: Supervoxel 1 is not star-shaped!
  Segment from SV Center Float32[10.987924, 15.309615, 18.614902] to Barycenter Float32[11.356835, 13.348973, 12.75096]
  (associated with triangle 5)
  intersects triangle 3 with vertices:
    v0: Float32[9.682824, 13.836763, 11.69176]
    v1: Float32[9.805282, 15.462804, 8.673942]
    v2: Float32[12.0708065, 13.082815, 14.606965]
-------------------------------------------
Displaying visualization of the error...
-------------------------------------------
Error: Supervoxel 1 is not star-shaped!
  Segment from SV Center Float32[10.987924, 15.309615, 18.614902] to Barycenter Float32[11.356835, 13.348973, 12.75096]
  (associated with triangle 5)
  intersects triangle 3 with vertices:
    v0: Float32[9.682824, 13.836763, 11.69176]
    v1: Float32[9.805282, 15.462804, 8.673942]
    v2: Float32[12.0708065, 13.082815, 14.606965]
-----------

TaskFailedException: TaskFailedException

    nested task error: InterruptException:
    Stacktrace:
      [1] poptask(W::Base.IntrusiveLinkedListSynchronized{Task})
        @ Base ./task.jl:1012
      [2] wait()
        @ Base ./task.jl:1021
      [3] wait(c::Base.GenericCondition{Base.Threads.SpinLock}; first::Bool)
        @ Base ./condition.jl:130
      [4] wait
        @ ./condition.jl:125 [inlined]
      [5] _trywait(t::Timer)
        @ Base ./asyncevent.jl:145
      [6] wait
        @ ./asyncevent.jl:162 [inlined]
      [7] sleep(sec::Float64)
        @ Base ./asyncevent.jl:272
      [8] sleep(timer::Makie.BudgetedTimer)
        @ Makie /usr/local/share/julia/packages/Makie/pFPBw/src/utilities/timing.jl:120
      [9] on_demand_renderloop(screen::GLMakie.Screen{GLFW.Window})
        @ GLMakie /usr/local/share/julia/packages/GLMakie/xG4T9/src/screen.jl:946
     [10] renderloop(screen::GLMakie.Screen{GLFW.Window})
        @ GLMakie /usr/local/share/julia/packages/GLMakie/xG4T9/src/

In [84]:
indicies_max=9
curr=[]
triangles=[]
batch_idx=1
start_idx=1
for index in [5,8]#start_idx:indicies_max
    tr=get_triangle_points(Array(tetr_dat_out), index, batch_idx)
    push!(triangles, tr)
end
sv_center_vec = get_sv_center(Array(tetr_dat_out)
, start_idx, batch_idx)
triangles=map(p-> Meshes.Triangle(Meshes.Point(Float64.(p[1])...),Meshes.Point(Float64.(p[2])...),Meshes.Point(Float64.(p[3])...)), triangles)

viz(triangles, color=1:length(triangles),alpha=0.5)#,alpha=0.5



# viz(curr_b, color=1:length(curr_b))#,alpha=0.5

TaskFailedException: TaskFailedException

    nested task error: InterruptException:
    Stacktrace:
      [1] poptask(W::Base.IntrusiveLinkedListSynchronized{Task})
        @ Base ./task.jl:1012
      [2] wait()
        @ Base ./task.jl:1021
      [3] wait(c::Base.GenericCondition{Base.Threads.SpinLock}; first::Bool)
        @ Base ./condition.jl:130
      [4] wait
        @ ./condition.jl:125 [inlined]
      [5] _trywait(t::Timer)
        @ Base ./asyncevent.jl:145
      [6] wait
        @ ./asyncevent.jl:162 [inlined]
      [7] sleep(sec::Float64)
        @ Base ./asyncevent.jl:272
      [8] sleep(timer::Makie.BudgetedTimer)
        @ Makie /usr/local/share/julia/packages/Makie/pFPBw/src/utilities/timing.jl:120
      [9] on_demand_renderloop(screen::GLMakie.Screen{GLFW.Window})
        @ GLMakie /usr/local/share/julia/packages/GLMakie/xG4T9/src/screen.jl:946
     [10] renderloop(screen::GLMakie.Screen{GLFW.Window})
        @ GLMakie /usr/local/share/julia/packages/GLMakie/xG4T9/src/

## visualization of "ridge"
We define ridge as sth that get from main oblique to the next main oblique in x,y, or z axis; here we visualize it for x axis 
In x axis we look into a ridge and triangles on both sides of it when one set of trianfles - blue is connected to lin_y and other to lin_z

In [85]:
main_index=0

max_index=(get_num_tetr_in_sv(false)*main_index)+11
min_index=(get_num_tetr_in_sv(false)*main_index)+1
sv_center_index=tetrs[min_index,1,1:4]
sv_centr=Array(sv_centers_out)[Int(sv_center_index[1]),Int(sv_center_index[2]),Int(sv_center_index[3]),:,1]
sv_centr_p_x=Array(sv_centers_out)[Int(sv_center_index[1]),Int(sv_center_index[2]-1),Int(sv_center_index[3]-1),:,1]
sv_centr_p_xy=Array(sv_centers_out)[Int(sv_center_index[1]),Int(sv_center_index[2]),Int(sv_center_index[3]-1),:,1]
sv_centr_p_y=Array(sv_centers_out)[Int(sv_center_index[1]),Int(sv_center_index[2])-1,Int(sv_center_index[3]),:,1]


barycenters_indicies=tetrs[min_index:max_index,5,1:4]
triangles_indicies=tetrs[min_index:max_index,2:4,:]

base_index=tetrs[min_index,2,1:4]
min_index


1

In [ ]:
using Meshes
using GLMakie
using LinearAlgebra # Ensure LinearAlgebra is loaded if not already
# using Makie: LabelElement # Removed this import

"""
    visualize_control_points_at_base(triangles_indicies, control_points_out, base_index, sv_centr, sv_centr_p_x, sv_centr_p_xy, sv_centr_p_y; batch_idx=1)

Extracts triangle definitions, constructs Meshes.Triangle objects,
and visualizes specific control point types (1-8) located at the `base_index`
within the `control_points_out` tensor. Also visualizes the triangles themselves,
the four provided supervoxel center points, and lines from the main SV center (`sv_centr`)
to the barycenter of each triangle.
The color of each triangle corresponds to its index in the input `triangles_indicies`.
A legend identifies the control point types and SV centers, and a label explains the triangle coloring.

Args:
    triangles_indicies (Array{<:Real, 3}): Tensor of size (num_triangles, 3, 4).
        Each [i, j, :] contains [ix, iy, iz, point_type] indices into `control_points_out`
        for the j-th vertex of the i-th triangle.
    control_points_out (Array{<:Real, 6}): Tensor containing control point coordinates.
        Expected dimensions: (nx, ny, nz, num_point_types, 3, num_batches).
    base_index (AbstractVector{<:Integer}): A vector/tuple of 3 integers [bx, by, bz]
        specifying the base location in `control_points_out` to visualize points 1-8 from.
    sv_centr (AbstractVector{<:Real}): Coordinates [x, y, z] of the main SV center.
    sv_centr_p_x (AbstractVector{<:Real}): Coordinates [x, y, z] of the SV center +x neighbor.
    sv_centr_p_xy (AbstractVector{<:Real}): Coordinates [x, y, z] of the SV center +x+y neighbor.
    sv_centr_p_y (AbstractVector{<:Real}): Coordinates [x, y, z] of the SV center +y neighbor.
    batch_idx (Int): The batch index to use (default: 1).

Returns:
    Tuple: (fig, ax, triangles, legend_points)
        - fig: The GLMakie Figure object.
        - ax: The GLMakie Axis3 object.
        - triangles: A vector of Meshes.Triangle objects.
        - legend_points: A dictionary mapping point names to Meshes.Point objects.
"""
function visualize_control_points_at_base(triangles_indicies, control_points_out, base_index, sv_centr, sv_centr_p_x, sv_centr_p_xy, sv_centr_p_y; batch_idx=1)
    println("Step 1: Defining point types and colors...")
    # Define mapping for point types 1-8 and the additional main_oblique_x
    point_type_map = Dict(
        1 => (name="lin_x", color=:blue),
        2 => (name="lin_y", color=:cyan),
        3 => (name="lin_z", color=:purple),
        4 => (name="main_oblique", color=:orange),
        5 => (name="oblique_x_a", color=:red),
        6 => (name="oblique_x_b", color=:darkred),
        7 => (name="oblique_x_c", color=:pink),
        8 => (name="oblique_x_d", color=:magenta),
        9 => (name="main_oblique_x", color=:darkorange4) # Added entry for main_oblique_x (using type 9 conceptually)
    )
    println("Point type map defined.")
    # --- Code from Step 1 ---
    println("\nStep 2: Extracting triangle vertex coordinates...")
    num_triangles = size(triangles_indicies, 1)
    triangle_vertex_coords_list = Vector{Vector{Vector{Float64}}}(undef, num_triangles) # Preallocate outer vector

    for i in 1:num_triangles
        current_triangle_points = Vector{Vector{Float64}}(undef, 3) # Preallocate inner vector
        for j in 1:3 # Iterate through 3 vertices of the triangle
            # Get indices [ix, iy, iz, point_type]
            idx_info = triangles_indicies[i, j, :]
            ix, iy, iz, point_type = Int.(round.(idx_info)) # Ensure integer indices

            # Basic bounds check (optional but recommended)
            if checkbounds(Bool, control_points_out, ix, iy, iz, point_type, :, batch_idx)
                 # Extract coordinates [x, y, z] for the vertex
                coords = control_points_out[ix, iy, iz, point_type, :, batch_idx]
                current_triangle_points[j] = Float64.(coords) # Store as Float64
            else
                println("Warning: Indices out of bounds for triangle $i, vertex $j: [$ix, $iy, $iz, $point_type]")
                current_triangle_points[j] = [NaN, NaN, NaN] # Placeholder for invalid points
            end
        end
        triangle_vertex_coords_list[i] = current_triangle_points
    end

    # --- Added Step 2.5 ---
    colors_by_original_index = Vector{Symbol}(undef, num_triangles)
    for i in 1:num_triangles
        has_lin_z = false
        has_lin_y = false
        for j in 1:3
            # Check bounds before accessing index - use NaN or skip if out of bounds
            if checkbounds(Bool, triangles_indicies, i, j, 4)
                point_type = Int(round(triangles_indicies[i, j, 4]))
                if point_type == 3 # lin_z
                    has_lin_z = true
                    break # Prioritize green
                elseif point_type == 2 # lin_y
                    has_lin_y = true
                end
            else
                 println("Warning: Index out of bounds when checking point type for triangle $i, vertex $j.")
                 # Decide default color or handle error - using gray here
                 has_lin_z = false
                 has_lin_y = false
                 break
            end
        end

        if has_lin_z
            colors_by_original_index[i] = :green
        elseif has_lin_y
            colors_by_original_index[i] = :red
        else
            colors_by_original_index[i] = :gray # Default color
        end
    end


# filepath: /workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/src/tests/points_from_weights/points_from_weights_viz.ipynb
    # --- Modified Step 3 ---
    triangles = Meshes.Triangle[] # Initialize empty vector
    final_triangle_colors = Symbol[] # Initialize empty vector for colors
    barycenters = Meshes.Point[] # Initialize empty vector for barycenters

    for i in 1:num_triangles # Iterate through original indices
        p_coords = triangle_vertex_coords_list[i] # Get coords extracted in Step 2
        # Check for NaN coordinates introduced by bounds errors
        if any(isnan, vcat(p_coords...))
            # Skip this triangle
        else
            p1 = Meshes.Point(p_coords[1]...)
            p2 = Meshes.Point(p_coords[2]...)
            p3 = Meshes.Point(p_coords[3]...)
            triangle = Meshes.Triangle(p1, p2, p3)
            push!(triangles, triangle) # Add valid triangle
            push!(final_triangle_colors, colors_by_original_index[i]) # Add corresponding color
            # Calculate and store barycenter
            barycenter = centroid(triangle) # Meshes.jl provides centroid function
            push!(barycenters, barycenter)
        end
    end
    # Note: Filter for dummy triangles is removed as we build the list directly

    # --- Code from Step 1, 2, 2.5 & 3 ---
    legend_points = Dict{String, Meshes.Point}()
    bx, by, bz = Int.(round.(base_index)) # Ensure integer indices

    max_point_type_to_visualize = 8 # Only visualize types 1 through 8 initially from the map
    for point_type in 1:max_point_type_to_visualize
        if haskey(point_type_map, point_type)
            name = point_type_map[point_type].name
             # Basic bounds check
            if checkbounds(Bool, control_points_out, bx, by, bz, point_type, :, batch_idx)
                coords = control_points_out[bx, by, bz, point_type, :, batch_idx]
                p = Meshes.Point(Float64.(coords)...)
                legend_points[name] = p
            else
                 println("Warning: Indices out of bounds for legend point type $point_type at base_index: [$bx, $by, $bz]")
            end
        else
             println("Warning: No definition found in point_type_map for point type $point_type.")
        end
    end

    # Extract the additional "main_oblique_x" point
    main_oblique_x_idx = (bx + 1, by, bz) # Index for the next main oblique along x
    main_oblique_type = 4 # Point type for main oblique
    main_oblique_x_name = "main_oblique_x"
    if checkbounds(Bool, control_points_out, main_oblique_x_idx..., main_oblique_type, :, batch_idx)
        coords_x = control_points_out[main_oblique_x_idx..., main_oblique_type, :, batch_idx]
        p_x = Meshes.Point(Float64.(coords_x)...)
        legend_points[main_oblique_x_name] = p_x
    else
        println("Warning: Indices out of bounds for $main_oblique_x_name at index: $main_oblique_x_idx")
    end

    println("Finished extracting legend points.")

    # --- Add SV Center Points ---
    sv_centr_point = Meshes.Point(Float64.(sv_centr)...) # Convert main center to Point
    sv_center_points = Dict(
        "sv_centr" => (point=sv_centr_point, color=:green),
        "sv_centr_p_x" => (point=Meshes.Point(Float64.(sv_centr_p_x)...), color=:lightgreen),
        "sv_centr_p_xy" => (point=Meshes.Point(Float64.(sv_centr_p_xy)...), color=:darkgreen),
        "sv_centr_p_y" => (point=Meshes.Point(Float64.(sv_centr_p_y)...), color=:springgreen)
    )
    # Add SV centers to the legend_points dictionary for potential later use
    for (name, data) in sv_center_points
        legend_points[name] = data.point
        println("  Added SV center point: $name")
    end
    println("Finished adding SV Center points.")

    # --- Modified Step 5 ---
    println("\nStep 5: Visualizing triangles, points, and segments...")
    GLMakie.activate!() # Ensure GLMakie is active
    fig = Figure(size = (1000, 700)) # Adjust figure size if needed
    # Modify the title - keep it concise
    ax = Axis3(fig[1, 1], aspect = :data, title = "Control Points at Base $base_index & SV Centers") # Updated title

    # Visualize the triangles themselves
    if !isempty(triangles)
        # Color triangles based on the determined colors
        viz!(ax, triangles, color=final_triangle_colors, alpha=0.7)
        println("  Added triangles to plot with specific colors.")
    end

    # Visualize the legend points and collect legend entries
    legend_elements = [] # Renamed from legend_entries for clarity
    legend_labels = []   # Separate list for labels

    # Visualize Control Points (1-8 and main_oblique_x)
    if !isempty(legend_points)
        # Iterate through defined point types including the conceptual type 9 for main_oblique_x
        for point_type in vcat(1:max_point_type_to_visualize, 9) # Include type 9
             if haskey(point_type_map, point_type)
                name = point_type_map[point_type].name
                color = point_type_map[point_type].color
                if haskey(legend_points, name)
                    point_to_plot = legend_points[name]
                    # Plot the point
                    viz!(ax, point_to_plot, color=color, pointsize=15) # No label needed here
                    # Create a MarkerElement for the legend
                    push!(legend_elements, MarkerElement(color = color, marker = :circle, markersize=15, markerstrokecolor=:black))
                    push!(legend_labels, name) # Store the label string

                    println("  Plotting control point: $name with color $color")
                end
            end
        end
    end

    # Visualize SV Center Points
    println("  Plotting SV Center points...")
    for (name, data) in sv_center_points
        viz!(ax, data.point, color=data.color, pointsize=20) # Larger size for SV centers
        push!(legend_elements, MarkerElement(color = data.color, marker = :dtriangle, markersize=20, markerstrokecolor=:black)) # Use triangle marker
        push!(legend_labels, name)
        println("    Plotting SV center: $name with color $(data.color)")
    end

    # --- Visualize lines from SV center to barycenters ---
    println("  Creating and visualizing lines to barycenters...")
    barycenter_lines = Meshes.Segment[]
    if !isempty(barycenters)
        for barycenter in barycenters
            # Check if barycenter is valid (not NaN)
            if !any(isnan, coordinates(barycenter))
                 push!(barycenter_lines, Meshes.Segment(sv_centr_point, barycenter))
            end
        end
        if !isempty(barycenter_lines)
            viz!(ax, barycenter_lines, color=:gray, linestyle=:dash, linewidth=1.5)
            println("    Added $(length(barycenter_lines)) lines from SV center to triangle barycenters.")
        end
    end

    # Create the Legend manually in the layout
    if !isempty(legend_elements)
            # Create Legend with elements and labels directly (removed nesting)
            Legend(fig[1, 2], legend_elements, legend_labels, "Points Legend", nbanks=2) # Updated title, Re-added nbanks=2
            println("  Added point legend to layout.")
    else
        println("  No legend points found or extracted to visualize.")
    end

    # Create and visualize the segments
    println("  Creating and visualizing segments...")
    segment_points = ["main_oblique", "oblique_x_a", "oblique_x_b",  "oblique_x_c", "oblique_x_d","main_oblique_x"]
    segments_to_draw = []
    all_segment_points_found = true
    for i in 1:(length(segment_points) - 1)
        p1_name = segment_points[i]
        p2_name = segment_points[i+1]
        if haskey(legend_points, p1_name) && haskey(legend_points, p2_name)
            p1 = legend_points[p1_name]
            p2 = legend_points[p2_name]
            push!(segments_to_draw, Meshes.Segment(p1, p2))
            println("    Created segment: $p1_name -> $p2_name")
        else
            println("Warning: Could not create segment $p1_name -> $p2_name. Point(s) missing.")
            all_segment_points_found = false
        end
    end

    if !isempty(segments_to_draw)
        viz!(ax, segments_to_draw, color=:black, linewidth=3) # Use linewidth for thickness
        println("  Added segments to plot.")
    end


     # Add a separate label explaining triangle colors below the legend
     Label(fig[2, 1:2], "Triangle color: Green (contains lin_z), Red (contains lin_y, no lin_z), Gray (otherwise).", tellwidth=false, halign = :center)
     println("  Added triangle color explanation label.")


    display(fig)
    println("Visualization complete.")

    return fig, ax, triangles, legend_points
end

# Make sure the input arrays are on the CPU
control_points_out_cpu = Array(control_points_out) # Assuming control_points_out might be a CuArray
triangles_indicies_cpu = Array(triangles_indicies) # Assuming triangles_indicies might be from tetrs (CPU)
base_index_cpu = Array(base_index) # Ensure base_index is a CPU array/vector
# Ensure SV centers are also CPU arrays
sv_centr_cpu = Array(sv_centr)
sv_centr_p_x_cpu = Array(sv_centr_p_x)
sv_centr_p_xy_cpu = Array(sv_centr_p_xy)
sv_centr_p_y_cpu = Array(sv_centr_p_y)


# Define the batch index you want to visualize
batch_to_visualize = 1

# Call the function with the new arguments
fig_legend, ax_legend, extracted_triangles, extracted_legend_points = visualize_control_points_at_base(
    triangles_indicies_cpu,
    control_points_out_cpu,
    base_index_cpu,
    sv_centr_cpu,          # Pass the SV center points
    sv_centr_p_x_cpu,
    sv_centr_p_xy_cpu,
    sv_centr_p_y_cpu,
    batch_idx=batch_to_visualize
)


using Logging # Add logging if not already present

"""
    print_triangle_point_names(triangles_indicies)

Prints the human-readable names of the point types forming each triangle.

Args:
    triangles_indicies (Array{<:Real, 3}): Tensor of size (num_triangles, 3, 4).
        Each [i, j, :] contains [ix, iy, iz, point_type] indices.
        The function uses only the point_type (the 4th element).
"""
function print_triangle_point_names(triangles_indicies)
    # Define mapping from point type index to human-readable name
    point_type_map = Dict(
        1 => "lin_x",
        2 => "lin_y",
        3 => "lin_z",
        4 => "main_oblique",
        5 => "oblique_x_a",
        6 => "oblique_x_b",
        7 => "oblique_x_c",
        8 => "oblique_x_d"
        # Add more mappings if other point types are possible
    )

    num_triangles = size(triangles_indicies, 1)
    println("Analyzing $num_triangles triangles:")

    for i in 1:num_triangles
        vertex_names = String[] # Store names for the current triangle
        valid_triangle = true
        for j in 1:3 # Iterate through the 3 vertices of the triangle
            # Extract the point type (the 4th element in the last dimension)
            # Ensure it's treated as an integer for dictionary lookup
            point_type = Int(round(triangles_indicies[i, j, 4]))

            # Look up the name in the map
            point_name = get(point_type_map, point_type, "Unknown Type ($point_type)")
            if point_name == "Unknown Type ($point_type)"
                 @warn "Triangle $i, Vertex $j: Found unknown point type $point_type"
                 valid_triangle = false # Mark triangle as potentially invalid if type is unknown
            end
            push!(vertex_names, point_name)
        end

        # Print the result for the current triangle
        println("Triangle $i: [$(join(vertex_names, ", "))]")
        # Optionally add more details if needed, like the raw indices
        # println("  Raw indices: $(triangles_indicies[i, :, :])")
    end
    println("Finished analyzing triangles.")
end

# Example Usage:
# Assuming 'triangles_indicies' is already defined and populated in your environment
# Make sure it's on the CPU if it came from GPU operations
triangles_indicies_cpu = Array(triangles_indicies)


# You can inspect the results
for (name, point) in extracted_legend_points
    println("- $name: $point")
end
print_triangle_point_names(triangles_indicies_cpu)


Step 1: Defining point types and colors...
Point type map defined.

Step 2: Extracting triangle vertex coordinates...
Finished extracting legend points.
  Added SV center point: sv_centr_p_x
  Added SV center point: sv_centr_p_y
  Added SV center point: sv_centr
  Added SV center point: sv_centr_p_xy
Finished adding SV Center points.

Step 5: Visualizing triangles, points, and segments...
  Added triangles to plot with specific colors.
  Plotting control point: lin_x with color blue
  Plotting control point: lin_y with color cyan
  Plotting control point: lin_z with color purple
  Plotting control point: main_oblique with color orange
  Plotting control point: oblique_x_a with color red
  Plotting control point: oblique_x_b with color darkred
  Plotting control point: oblique_x_c with color pink
  Plotting control point: oblique_x_d with color magenta
  Plotting control point: main_oblique_x with color darkorange4
  Plotting SV Center points...
    Plotting SV center: sv_centr_p_x with

UndefVarError: UndefVarError: `coordinates` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing GeometryBasics in the current active module Main
Hint: a global variable of this name also exists in Contour.

In [87]:
# todo probably the root issue in this example here is that in order for it to work the ridge that gets from main oblique to next main oblique in x throgh 
#     additional obliques  needs to be always between lin_z and lin_y 

In [88]:
# Probably solution woulb be to constraint the derivation of y and z  component of points between oblique x to be between just  
# lin_z, lin_y
# x komponents are contrained just by main obliques 
# - of course we need to adapt this in axes for oblique y and z 